In [0]:
customer_df=(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format","json")
    .option("cloudFiles.schemaLocation","/Volumes/gizmobox/landing/operational_data/customer_autoloader/_schema")
    .option("cloudFiles.inferColumnTypes","true")
    .load("/Volumes/gizmobox/landing/operational_data/customer_autoloader/")
)



In [0]:
customer_df=(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format","json")
    .option("cloudFiles.schemaLocation","/Volumes/gizmobox/landing/operational_data/customer_autoloader/_schema")
    .option("cloudFiles.inferColumnTypes","true")
    .option("cloudFiles.schemaHints","date_of_birth DATE, member_since DATE,created_timestamp TIMESTAMP")
    .load("/Volumes/gizmobox/landing/operational_data/customer_autoloader/")
)



In [0]:
from pyspark.sql.functions import col, current_timestamp
customer_transformed_df = (
    customer_df.withColumn("file_name", col("_metadata.file_name"))
    .withColumn("ingestion_date", current_timestamp())
)
display(customer_transformed_df)

In [0]:
streaming_query = (
                    customer_transformed_df.writeStream
                    .format("delta")
                    .option("checkpointLocation","/Volumes/gizmobox/landing/operational_data/customer_autoloader/_checkpoint_stream"
    )
    .toTable("gizmobox.bronze.customer_autoloader")
)

In [0]:
%sql
select count(*) from gizmobox.bronze.customer_autoloader

In [0]:
streaming_query.stop()